MY APPROACH

STEP 1:
Finding the books which user has readed and the books which has same authors and score them (scoring system : book read by user =1 , book which has same author +=0.25).

STEP 2:
Finding the top users which has similar taste in books by aggrating the sum of score from the score of books (in simple words finding the user score by how many book they have read that have more that 0 score and summing it) 
For example:
suppose user A has read 80 books in total from which 36 books has 0 score, 40 books has 0.25 score and 4 books has 1.25 score so user A total_score is 40 * 0.25 + 4 * 1.25 which is equal to 15.

STEP 3:
finding the books which is read by most users based on their score (in simple words books are scored by how many users has read them * by their score respictively)
For example:
suppose book K is read by 38 users from which 8 have score of 0 ,10 has score of 6 and 20 has score of 5 then book_score is 160

In [46]:
import pandas as pd

In [47]:
df_book= pd.read_csv('Books.csv' , usecols=['ISBN','Book-Title','Book-Author','Year-Of-Publication'],dtype={'Year-Of-Publication':'int16'})
# to remove duplcate entries based on book-title
df_book = df_book.drop_duplicates(subset='Book-Title').reset_index(drop=True)
df_user=pd.read_csv('Ratings.csv' , usecols=['User-ID','ISBN'])

# making sure the column ISBN always has 10 character if it has less than 10 character then filling the mising character as 0 from left side
df_book['ISBN'] = df_book['ISBN'].astype(str).str.zfill(10)

In [ ]:
# taking the list of books the user has already read
my_book=["The Testament","Dead Aim","The Deal"]

# finding the list of their authors 
books_author=df_book.loc[df_book['Book-Title'].isin(my_book),'Book-Author'].unique()

# assigning scores to books
df_book['score']=0.0
df_book.loc[df_book['Book-Title'].isin(my_book),'score'] = 1
df_book.loc[df_book['Book-Author'].isin(books_author),'score'] += 0.25

# selecting books with score grater that 0
df_book_score= (
    df_book[df_book['score'] > 0]
    .sort_values(by='score' , ascending=False)
    .copy()
)

(step 1) score for book is completed

In [49]:
# merging of df_book_score & df_user for making a df with both ISBN and User-ID
df_user_score= pd.merge(df_book_score, df_user, how='inner', on='ISBN').sort_values(by='score',ascending=False)

In [50]:
# obtaning user score
user_total_score = (
    df_user_score.groupby('User-ID')                        
    .agg(
        total_score=('score', 'sum'),
        rating_count=('score', 'count')
    )
    .sort_values(by=['total_score','rating_count'], ascending=False)
    .reset_index()
)
# getting the users which has more than or equal to 2 score
top_users= user_total_score[user_total_score['total_score']>=2].head(800)

(step 2) finding the like minded users is completed

In [51]:
# merginf dfs for getting df which has User-Id , ISBN and total_score of user
merged_df= (
    pd.merge(top_users,df_user_score,how='inner',on='User-ID')
    .sort_values(by='total_score',ascending=False)
    .reset_index(drop=True)
    [['User-ID','total_score','ISBN']]
)


In [52]:
# for finding final book score
grouped= (
    merged_df.groupby('ISBN')
    .agg(
        book_score= ('total_score','sum'),
        total_count= ('total_score','count')
    )
    .sort_values(by='book_score',ascending=False)
    .reset_index()
)

In [53]:
# merging for getting Book-Title
final_grouped= pd.merge(grouped,df_book_score,how='inner',on='ISBN')[['ISBN', 'Book-Title', 'book_score','Book-Author']]

# getting top 5 rated books for suggesting excluding the books already read by the user
top_5_books=(
    final_grouped[~final_grouped['Book-Title'].isin(my_book)]
    .reset_index(drop=True)
    .head(5)
)

In [54]:
# getting the final answer as list which contains the list of book title and book score
top_5_books[['Book-Title','book_score']].values.tolist()

[['Dark Rider', 9.0],
 ['Long After Midnight', 8.75],
 ['Body of Lies', 6.75],
 ['The Beloved Scoundrel', 4.25],
 ['Midnight Warrior', 4.25]]